# AI as a Judge for Prompt Modification
Use AI as a judge to modify a prompt to navigate it through a 'maze'

In [61]:
import sys
import os
import dspy 

import dspy 
from pydantic import BaseModel
from typing import Literal

from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


In [62]:
import dspy

# # Configure DSPy to use the Responses API for your language model
# dspy.settings.configure(
#     lm=dspy.LM(
#         "openai/gpt-5-mini",
#         model_type="responses",
#         temperature=1.0,
#         max_tokens=16000,
#     ),
# )


lm = dspy.LM(
    'gpt-4.1', 
    model_type='chat', 
    cache=False, 
    api_key=settings.OPENAI_API_KEY,
    temperature=0.8     
)

dspy.configure(lm=lm)

In [ ]:

################################################################
# Setup initial order
# widget = Widget(shape="round", weightInKg=2)
# order = WidgetOrder(widget=widget, order_placement_instructions="Place an order")

# print(order)

In [ ]:
class Widget(BaseModel):
    shape: Literal["round", "square", "triangle"] = dspy.OutputField()
    weightInKg: int = dspy.OutputField()

class WidgetOrder(dspy.Signature):
    """
    Follow the ```order_placement_instructions``` to place an order for a widget. 
    The ```original_widget` field contains the details of the widget to be ordered.
    """

    order_placement_instructions: str = dspy.InputField()
    original_widget: Widget = dspy.InputField()

    widget: Widget = dspy.OutputField()
    

################################################################
# Function to place order

def place_order(order: WidgetOrder) -> str:
    """
    Places an order for widgets.

    Return: 
    - WidgetOrder if the order was NOT placed successfully the orderRejectedReason field is populated with the reason for rejection. 
    """

    if order.widget.shape == "round" and order.widget.weightInKg <= 5:
        return "Round widgets cannot weigh less than or equal to 5kg. Try again with a different shape or weight."

    return "Success"


order_placement_instructions='Place an order' orderRejectedReason=None widget=Widget(shape='round', weightInKg=2)


In [ ]:
create_order = dspy.Predict(WidgetOrder)

################################################################
# Place order 

loopCount = 0
order_placement_instructions = "Create a round widget that weighs 2kg."

while loopCount < 2:
    md("loopCount: ", loopCount)
    order = create_order(order_placement_instructions, 
     = place_order(order)

    md('**Order was rejected for reason**: ', response.orderRejectedReason)
    response.order_placement_instructions = response.orderRejectedReason + " Please amend the order accordingly."
    md("**Response order**: ", response)
    order = amend_order(
        order_placement_instructions=response.order_placement_instructions,
        widget=order.widget,
        orderRejectedReason=response.orderRejectedReason)
    
    #md("Amended order**: ", order)

    loopCount += 1

loopCount: 0

AttributeError: 'Prediction' object has no attribute 'widget'